In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import matplotlib.pyplot as plt

from ai.TrashClassifierAE import TrashClassifier
from ai.DataLoader2 import DataLoader
from env import *

EPOCHS = 100
ETA = 1e-3

In [7]:
model = TrashClassifier(fine_tune=False)
model = model.cuda()

FeatureAE was loaded.


In [8]:
train_dloader = DataLoader(TRASH_TRAIN_DATA_PATH, TRASH_CAT)
valid_dloader = DataLoader(TRASH_VALID_DATA_PATH, TRASH_CAT)

Number of data batch: 324
Number of batch: 5


In [9]:
optimizer = optim.Adam(model.parameters(), lr=ETA)
criterion = nn.NLLLoss()

In [11]:
for e in range(EPOCHS):
    
    train_loss = 0.0
    train_acc = 0.0
    valid_loss = 0.0
    valid_acc = 0.0
    
    for x_batch, y_batch in train_dloader.next_batch():
        x_batch = torch.FloatTensor(x_batch).cuda()
        y_batch = torch.LongTensor(y_batch).cuda()
        
        logps = model(x_batch)
        loss = criterion(logps, y_batch)
        
        with torch.no_grad():
            train_loss += loss.item()
            ps = torch.exp(logps)
            
            ps_k, cls_k = ps.topk(1, dim=1)
            equal = cls_k == y_batch.view(*cls_k.size())
            train_acc += torch.mean(equal.type(torch.FloatTensor))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    if (e+1) % 10 == 0:
        with torch.no_grad():
            model.eval()

            for x_batch, y_batch in valid_dloader.next_batch():
                x_batch = torch.FloatTensor(x_batch).cuda()
                y_batch = torch.LongTensor(y_batch).cuda()

                logps = model(x_batch)
                loss = criterion(logps, y_batch)

                valid_loss += loss.item()

                ps = torch.exp(logps)
                ps_k, cls_k = ps.topk(1, dim=1)
                equal = cls_k == y_batch.view(*cls_k.size())
                valid_acc += torch.mean(equal.type(torch.FloatTensor))

            train_loss /= len(train_dloader)
            train_acc /= len(train_dloader)
            valid_loss /= len(valid_dloader)
            valid_acc /= len(valid_dloader)

            print(f"Epochs: {e+1}/{EPOCHS}")
            print(f"Train loss: {train_loss:.8f}")
            print(f"Train acc: {train_acc:.8f}")
            print(f"Valid loss: {valid_loss:.8f}")
            print(f"Valid acc: {valid_acc:.8f}")

            model.train()

Epochs: 10/100
Train loss: 0.22118256
Train acc: 0.94375002
Epochs: 20/100
Train loss: 0.11333561
Train acc: 0.98124999
Epochs: 30/100
Train loss: 0.10609844
Train acc: 0.98124999
Epochs: 40/100
Train loss: 0.07796553
Train acc: 0.98750001
Epochs: 50/100
Train loss: 0.10940139
Train acc: 0.96875000
Epochs: 60/100
Train loss: 0.10768088
Train acc: 0.96875000
Epochs: 70/100
Train loss: 0.03618611
Train acc: 0.98750001
Epochs: 80/100
Train loss: 0.08418716
Train acc: 0.97500002
Epochs: 90/100
Train loss: 0.03383254
Train acc: 0.99374998
Epochs: 100/100
Train loss: 0.05401335
Train acc: 0.98750001


In [12]:
model.save(CLF_CKPT_PATH)

Trash Classifier was saved.
